# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [2]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

# Install pdfplumber, a dependency for PDF processing in the agent's library
# if importlib.util.find_spec("pdfplumber") is None:
#     !pip install pdfplumber
    
    # # Only needed for Udacity workspace

# import importlib.util
# import sys

# # Check if 'pysqlite3' is available before importing
# if importlib.util.find_spec("pysqlite3") is not None:
#     import pysqlite3
#     sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
# Import necessary libraries for stateful agent
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from tavily import TavilyClient
from dotenv import load_dotenv
from openai import OpenAI
from typing import Optional

# Import the agent framework
# from lib.agents import Agent

In [4]:
# Load environment variables
load_dotenv()

# Validate required API keys with helpful error messages
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError(
        'OPENAI_API_KEY not found in environment variables. '
        'Please create a .env file with OPENAI_API_KEY="your_key"'
    )

tavily_api_key = os.getenv('TAVILY_API_KEY')
if not tavily_api_key:
    raise ValueError(
        'TAVILY_API_KEY not found in environment variables. '
        'Please create a .env file with TAVILY_API_KEY="your_key"'
    )

# Initialize clients with error handling
try:
    client = OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key=openai_api_key,
    )
except Exception as e:
    raise ValueError(f'Failed to initialize OpenAI client: {str(e)}')

try:
    tavily_client = TavilyClient(api_key=tavily_api_key)
except Exception as e:
    raise ValueError(f'Failed to initialize Tavily client: {str(e)}')

print('API clients initialized successfully!')

API clients initialized successfully!


In [5]:
from pydantic import BaseModel, Field
from typing import Optional

class AgentAnswer(BaseModel):
    answer: str = Field(description="The direct answer to the user's question.")
    source: str = Field(description="The source of the information, either the game's name or 'Web Search'.")
    fallback_used: bool = Field(description="True if a web search was required to answer the question.")
    internal_reasoning: str = Field(description="The agent's step-by-step reasoning for its conclusion.")

# Import the agent framework and tooling
from lib.agents import Agent
from lib.tooling import tool

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [6]:
@tool
def retrieve_game_info(query: str, n_results: int = 3) -> list[str]:
    """
    Semantic search: Finds most relevant games in the vector database.
    
    Args:
        query: A question about the game industry
        n_results: Number of results to return (default: 3)
    
    Returns:
        List of JSON strings containing game metadata including:
        - Platform: Game Boy, PlayStation 5, Xbox 360, etc.
        - Name: Name of the game
        - YearOfRelease: Year when the game was released
        - Description: Additional details about the game
        - Genre: Game genre
        - Publisher: Game publisher
    """
    print(f"🔍 Tool: Retrieving game info for query: '{query}'")
    chroma_client = chromadb.PersistentClient(path="chromadb")

    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        api_base="https://openai.vocareum.com/v1",  # For Vocareum
        model_name="text-embedding-ada-002"
    )

    collection = chroma_client.get_collection(
        name="udaplay",
        embedding_function=embedding_fn
    )

    # Query the collection
    results = collection.query(query_texts=[query], n_results=n_results)

    # Format the retrieved documents for the next step
    retrieved_docs = []
    if results and results['metadatas'][0]:
        for meta in results['metadatas'][0]:
            # Pass the full JSON metadata as a string
            retrieved_docs.append(json.dumps(meta))
    
    print(f"Retrieved {len(retrieved_docs)} games from vector database")
    return retrieved_docs

#### Evaluate Retrieval Tool

In [7]:
@tool
def evaluate_retrieval(query: str, context: list[str]) -> bool:
    """
    Based on the user's question and retrieved documents, analyzes if the documents 
    are sufficient to answer the question.
    
    Args:
        query: Original question from user
        context: Retrieved documents most similar to the user query in the Vector Database
    
    Returns:
        bool: True if documents are sufficient, False if web search is needed
    """
    print(f"Evaluating if retrieved context is sufficient for: '{query}'")
    
    # A simple but effective prompt for the LLM
    prompt = f"""
    Based *only* on the provided context below, can you confidently answer the following user question?
    Respond with only "yes" or "no".

    User Question: "{query}"

    Context:
    ---
    {context}
    ---
    """

    # Use the configured client object here
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=5,
        temperature=0.0
    )

    decision = response.choices[0].message.content.strip().lower()
    print(f"Evaluation: LLM decision is '{decision}'")
    return "yes" in decision

#### Game Web Search Tool

In [8]:
@tool
def game_web_search(query: str) -> str:
    """
    Performs a web search using Tavily API when local database is insufficient.
    
    Args:
        query: A question about the game industry
    
    Returns:
        str: Content from the most relevant web search result
    """
    print(f"🌐 Tool: Performing web search for query: '{query}'")
    try:
        response = tavily_client.search(query=query, search_depth="basic")
        # We'll return the most relevant search result content
        content = response['results'][0]['content']
        print(f"Web search completed successfully")
        return content
    except Exception as e:
        print(f"Error during Tavily search: {e}")
        return "Web search failed."

### Agent

In [9]:
# Test the tools individually to verify they work before agent integration
print("Testing retrieve_game_info:")
results = retrieve_game_info("Nintendo games")
print(f"Retrieved {len(results)} results")

print("\nTesting evaluate_retrieval:")
evaluation = evaluate_retrieval("What Nintendo games are available?", results)
print(f"Evaluation result: {evaluation}")

if not evaluation:
    print("\nTesting game_web_search:")
    web_result = game_web_search("popular Nintendo games")
    print(f"Web search result preview: {web_result[:200]}...")

# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

Testing retrieve_game_info:
🔍 Tool: Retrieving game info for query: 'Nintendo games'
Retrieved 0 games from vector database
Retrieved 0 results

Testing evaluate_retrieval:
Evaluating if retrieved context is sufficient for: 'What Nintendo games are available?'
Evaluation: LLM decision is 'no'
Evaluation result: False

Testing game_web_search:
🌐 Tool: Performing web search for query: 'popular Nintendo games'
Web search completed successfully
Web search result preview: Top Ten Nintendo Games · 1. OoT · 2. Super Mario 64 · 3. Metriod Prime · 4. Super Mario Galaxy · 5. Pokemon (series) · 6. Twilight Priincess · 7...


In [10]:
# Define the system prompt for UdaPlay agent
UDAPLAY_SYSTEM_PROMPT = """
You are UdaPlay, an AI research agent specialized in the video game industry.

Your capabilities include:
1. Searching a comprehensive database of video games using semantic search
2. Evaluating whether retrieved information is sufficient to answer queries
3. Performing web searches when local knowledge is insufficient

When answering questions:
1. First, search your local database for relevant game information
2. Evaluate if the retrieved context is sufficient to answer the question
3. If not sufficient, perform a web search for additional information
4. Provide comprehensive, accurate answers based on available information

Always be helpful, accurate, and cite your sources when possible.
For queries about game releases, platforms, genres, or specific game details,
prioritize information from your local database first.

Always structure your final response as valid JSON using the AgentAnswer format:
{
  "answer": "The direct answer to the user's question",
  "source": "Source of information (game name or 'Web Search')",
  "fallback_used": true/false,
  "internal_reasoning": "Step-by-step reasoning process"
}
"""

# Instantiate the stateful Agent
udaplay_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=UDAPLAY_SYSTEM_PROMPT,
    tools=[retrieve_game_info, evaluate_retrieval, game_web_search],
    temperature=0.0,
)

print('UdaPlay Agent instantiated successfully!')
print(f'Model: {udaplay_agent.model_name}')
print(f'Number of tools: {len(udaplay_agent.tools)}')
print(f'Available tools: {[tool.name for tool in udaplay_agent.tools]}')

UdaPlay Agent instantiated successfully!
Model: gpt-4o-mini
Number of tools: 3
Available tools: ['retrieve_game_info', 'evaluate_retrieval', 'game_web_search']


In [11]:
# Demonstrate memory and session management
# The agent maintains conversation state - let's test this with follow-up questions

print("=== Demonstrating Stateful Memory ===")

# First question
first_query = "Tell me about Nintendo games in our database"
print(f"Query 1: {first_query}")

run1 = udaplay_agent.invoke(first_query)
state1 = run1.get_final_state()
print(f"Answer 1: {state1['messages'][-1].content}")
print(f"Messages in session: {len(state1['messages'])}")

# Follow-up question that references the previous conversation
followup_query = "Which of those were released before 2000?"
print(f"\nQuery 2 (follow-up): {followup_query}")

run2 = udaplay_agent.invoke(followup_query)
state2 = run2.get_final_state()
print(f"Answer 2: {state2['messages'][-1].content}")
print(f"Messages in session: {len(state2['messages'])}")

# Demonstrate session reset
print("\n=== Session Management ===")
print(f"Session runs before reset: {len(udaplay_agent.get_session_runs())}")

# Start a new session
new_session_query = "What racing games do we have?"
run3 = udaplay_agent.invoke(new_session_query, session_id="racing_session")
state3 = run3.get_final_state()
print(f"New session answer: {state3['messages'][-1].content}")

# Show that different sessions have different histories
print(f"Default session runs: {len(udaplay_agent.get_session_runs('default'))}")
print(f"Racing session runs: {len(udaplay_agent.get_session_runs('racing_session'))}")

=== Demonstrating Stateful Memory ===
Query 1: Tell me about Nintendo games in our database
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🔍 Tool: Retrieving game info for query: 'Nintendo'
Retrieved 0 games from vector database
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
🌐 Tool: Performing web search for query: 'Nintendo games list'
Web search completed successfully
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer 1: {
  "answer": "Nintendo has a rich library of games, including classics like 'The Legend of Zelda' (1986), 'Super Mario 64 DS' (2004), and more recent titles like 'WarioWare: Get It Together!' (2021). Some notable games include 'Super Mario Galaxy 2' (2010) and 'Super Mario Maker 2' (2019).",
  "source": "Web Search",
  "fallback_used": true,
  "internal_reason

In [15]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?


# First question
first_query = "When Pokémon Gold and Silver was released?"
print(f"Query 1: {first_query}")

run1 = udaplay_agent.invoke(first_query)
state1 = run1.get_final_state()
print(f"Answer 1: {state1['messages'][-1].content}")



# Second question
second_query = "Which one was the first 3D platformer Mario game?"
print(f"Query 2: {second_query}")

run2 = udaplay_agent.invoke(second_query)
state2 = run2.get_final_state()
print(f"Answer 2: {state2['messages'][-1].content}")


# Third question
query = "Was Mortal Kombat X realeased for Playstation 5?"
print(f"Query 3: {query}")

run = udaplay_agent.invoke(query)
state = run.get_final_state()
print(f"Answer 3: {state['messages'][-1].content}")


# Fourth question
query = "when was Grand Turismo 5 released?"
print(f"Query 4: {query}")

run = udaplay_agent.invoke(query)
state = run.get_final_state()
print(f"Answer 4: {state['messages'][-1].content}")




Query 1: When Pokémon Gold and Silver was released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer 1: {
  "answer": "Pokémon Gold and Silver were released on October 15, 2000.",
  "source": "Web Search",
  "fallback_used": false,
  "internal_reasoning": "The information was previously retrieved from a web search, confirming the release date of Pokémon Gold and Silver."
}
Query 2: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer 2: {
  "answer": "The first 3D platformer Mario game is 'Super Mario 64'.",
  "source": "Web Search",
  "fallback_used": false,
  "internal_reasoning": "The information was previously retrieved from a web search, confirming that 'Super Mario 64' is the first 3D pl

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes